In [1]:
from sklearn.linear_model import LogisticRegressionCV, LassoLarsCV
import matplotlib.pyplot as plt
import pandas as pd
from gneiss.util import match
import seaborn as sns
import numpy as np
from gneiss.util import match
from scipy.stats import spearmanr

%matplotlib inline

In [2]:
fname = 'Full_table_new_metadata_N_acyls_features.xlsx'
metadata = pd.read_excel(fname, sheetname='Full_table_new_metadata_N_acyls') 
x1 = pd.read_excel(fname, sheetname='330.26')
x2 = pd.read_excel(fname, sheetname='344.28')
x3 = pd.read_excel(fname, sheetname='387.33')
x4 = pd.read_excel(fname, sheetname='401.33')
x5 = pd.read_excel(fname, sheetname='611.53')

In [3]:
x1 = x1.set_index('X.SampleID.1').iloc[:, 1:].dropna(1)
x2 = x2.set_index('X.SampleID.1').iloc[:, 1:].dropna(1)
x3 = x3.set_index('X.SampleID.1').iloc[:, 1:].dropna(1)
x4 = x4.set_index('X.SampleID.1').iloc[:, 1:].dropna(1)
x5 = x5.set_index('X.SampleID.1').iloc[:, 1:].dropna(1)

In [4]:
X = pd.concat((x1, x2, x3, x4, x5), axis=1)

In [5]:
bmi = metadata[['X.SampleID.1', 'BMI']]
bmi = bmi.set_index('X.SampleID.1')
bmi = bmi.dropna()
X, bmi = match(X, bmi)

In [6]:
varz = metadata.columns[2:496] 
pvals = {}

for v in varz:
    y = metadata[['X.SampleID.1', v]]
    y = y.set_index('X.SampleID.1')
    y = y.dropna()
    try:
        X_, y = match(X, y)
        y = np.ravel(y)
        if np.issubdtype(y.dtype, np.number):
            r = pd.DataFrame({
                'rho': [spearmanr(y, X_[c])[0] for c in X_.columns],
                'pvalue': [spearmanr(y, X_[c])[1] for c in X_.columns]
            })
            pvals[v] = np.min(r.pvalue)
            r.to_csv('correlations/%s.csv' % v)
    except:
        continue

/Users/mortonjt/miniconda3/envs/q2-gneiss/lib/python3.5/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/mortonjt/miniconda3/envs/q2-gneiss/lib/python3.5/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/mortonjt/miniconda3/envs/q2-gneiss/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/mortonjt/miniconda3/envs/q2-gneiss/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/mortonjt/miniconda3/envs/q2-gneiss/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [7]:
p = pd.Series(pvals)
p.to_csv('pvalues.csv')